In [1]:
# public packages
import andes
import ams
import cvxpy as cp
import numpy as np

# local packages
import anci_fun as af

# Para preparetion

In [2]:
# neural network parameters 

nn_name = 'trained_model.pth'

nn_weights, nn_bias, nn_diam = af.extract_nn_para(nn_name)

# need dimensions of the neural network to define linization variables

In [3]:
nn_weights

[array([[-0.09400599,  0.00255939, -0.03851063, ..., -0.18630776,
         -0.09999983, -0.1122928 ],
        [ 0.00503437,  0.02254246, -0.48973235, ..., -0.15373844,
         -0.23572348,  0.06699305],
        [-0.12874661,  0.20608553, -0.02862953, ...,  0.02541382,
         -0.02446124, -0.22243445],
        ...,
        [ 0.08542189,  0.43662688, -0.11580341, ..., -0.15186772,
         -0.01133465,  0.1691861 ],
        [ 0.14583746,  0.33122721, -0.32046276, ...,  0.11278176,
          0.00154687, -0.10406776],
        [ 0.17843458, -0.241758  ,  0.28884768, ..., -0.14984824,
          0.04582042, -0.02602023]]),
 array([[ 2.13471949e-02, -4.22798301e-04,  5.15808279e-05, ...,
          3.25602666e-03,  6.73401519e-04,  6.23292886e-02],
        [ 1.90471988e-02,  1.97596490e-01,  1.06377788e-02, ...,
          6.26905188e-02,  1.39970243e-01, -4.46147658e-02],
        [-3.17698866e-02, -4.80886638e-01, -1.34073094e-01, ...,
         -1.23209981e-02, -3.80469225e-02, -2.30095461e-

In [4]:
nn_diam

[27, 512, 6]